# Environment Setting

In [1]:
import aie

# 使用token进行鉴权
aie.Authenticate(token='ff0859af79711070b75b93848452399e')

aie.Initialize()

work_dir = "./work_dirs/tutorial"   

计算资源初始化中，请等待...
计算资源初始化完成.


# Dataset

In [ ]:
pip install torch
pip install scikit-image

In [7]:
pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install mmcv

  Preparing metadata (setup.py) ... done
  Using cached rich-13.9.2-py3-none-any.whl.metadata (18 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1208 lines of output]
      /tmp/pip-install-dknso9wl/mmcv_ad74d5c346e24a4b93fa16d38557f4e3/setup.py:5: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
        from pkg_resources import DistributionNotFound, get_distribution, parse_version
      running bdist_wheel
      /root/.conda/envs/hydro/envs/aie-sdk/lib/python3.11/site-packages/setuptools/_distutils/cmd.py:111: SetuptoolsDeprecationWarning: bdist_wheel.universal is deprecated
      !!
      
      

In [11]:
# 下载样本集
from aiearth.deeplearning.cloud.datasets import LandcoverDataset, PublicDatasetMeta
gid_15_train_dataset = LandcoverDataset(PublicDatasetMeta.GID_15_TRAIN["dataset_id"], data_root=work_dir)

ModuleNotFoundError: No module named 'mmcv'

In [ ]:
from aiearth.deeplearning.sampler import RandomNonGeoDatasetSampler
# 随机按照80%， 20%进行切分成两个新样本集
train_dataset, val_dataset = RandomNonGeoDatasetSampler.split_by_percent(gid_15_train_dataset, 0.8)
# 随机提取20张图生成新样本集
test_dataset = RandomNonGeoDatasetSampler.sample_by_count(gid_15_train_dataset, 20) 

# Model

In [ ]:
# model
from aiearth.deeplearning.trainer.mmseg import LandcoverTrainer

trainer = LandcoverTrainer(work_dir=work_dir, config_name="fcn_hr48_1024x1024_16k_landcover")

# Trainer Setting

In [ ]:
# 设置样本集
trainer.setup_datasets(train_dataset)
trainer.setup_datasets(val_dataset, data_type="val")
trainer.setup_datasets(test_dataset, data_type="test")

# 设置pretrained model
from aiearth.deeplearning.model_zoo.model import PretrainedModel
model = PretrainedModel("aie://LandCover/landcover_v1.6.pth")
trainer.load_from(model.local_path)

# 设置训练参数
trainer.cfg.runner["max_iters"] = 2  
trainer.cfg.checkpoint_config["interval"]=1
trainer.cfg.data.samples_per_gpu = 1

In [ ]:
# 训练
trainer.train(validate=False)

In [ ]:
import os
# 保存的训练checkpoint路径
checkpoint = os.path.join(work_dir, "latest.pth")

# Test

In [ ]:
trainer.test(checkpoint, output_dir="test_ret")

# Onnx Export

In [ ]:
onnx_path = checkpoint.replace(".pth", ".onnx")

trainer.export_onnx(output_file=onnx_path, checkpoint_path=checkpoint,shape=(1024, 1024))